<dl>
<table width="225px">
  <tr>
    <td width="52px"><img src="./kernels/sparkr/logo-64x64.png" alt="H2O R" style="width:48px;height:48px;"></td>
    <td bgcolor="#EFEEEC"><h1><font color="#EFEEEC">__</font>  SparkR</h1></td>
  </tr>
</table>
</dl>
**************

Apache Spark is an open source cluster computing framework. Unlike Hadoop's two-stage disk-based MapReduce paradigm, Spark uses multi-stage in-memory primitives. These primatives can be managed in a couple ways, we'll be using YARN. The following script will create an SparkContext, sc, which will then initiate a Spark Application using YARN as the cluster manager. We will use the R Spark API, more commonly known as SparkR.

# Invoke Spark-On-YARN Application using R API (SparkR):

In [ ]:
source('../init/spark_init.r')

**************
# Verify Spark-On-YARN Application is Running:
Check the status of your application, named **SparkR**, within the [YARN RUNNING Applications](http://localhost:8088/cluster/apps/RUNNING) List.

**************
# Create DataFrame Using HiveContext
A DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data.frame in localized R. SparkR DataFrames can be constructed from two sources: 
   * **Tables in Hive**
   * Structured data files (from local FS or HDFS)

We will use an existing Hive table, dat. We generated and loaded this data from script; we know it has the following properties:

$$ y = \beta_0 + \sum_{i=1}^{3} \beta_i x_i$$
$$ \beta_0: 10 $$
$$ \beta_1:  3 $$
$$ \beta_2: -2 $$
$$ \beta_3: -1 $$

In [ ]:
dat <- sql(sqlContext, "SELECT y, x1, x2, x3 FROM dat")
printSchema(dat)

# Train model on DataFrame
Unlike pySpark, we do not need to take the extra step to change the record form into LabeledPoint. We can proceed directly to calling the familiar glm function, including the use of R [formula](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/formula.html). 

In [ ]:
f <- formula("y ~ x1 + x2 + x3")
model <- glm(f,data=dat)
summary(model)

If everything ran as expected, we should see model weights that estimate the true weights used to generate our dataset:

| **$coeffients**      | **True**     |
| ------------- |:-------------:|
| **(Intercept)**      | 10.0 |
| x1    | 3.0     |
| x2 | -2.0      |
| x3 | -1.0 |

### SparkR is not Distributed R
For existing R users, this interface is very appealing. However, there are a couple clarifications that should be made on the familiar [glm](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/glm.html) function. Since the class of data is found to be a spark DataFrame, the function calls instead to MLlib. The R API for spark makes all the form changes necessary use MLlib.
#### Limited functions list
Ok, quite limited as of writing. The only R distributed ML function is [glm](https://spark.apache.org/docs/1.5.0/sparkr.html#machine-learning).
#### R Parallelizability
Without a doubt, the breadth of R libraries with mathmatical functions make the language extremely appealing for data scientists. We should realize though that many of algorithms written in R will not be parallelizable for spark and as a result sparkR will grow to include only [parallelizable functions](https://spark.apache.org/docs/1.5.0/ml-guide.html#algorithm-guides) and not the entire realm of functions in [CRAN R](https://cran.r-project.org/).

# Stop Spark-On-YARN Application:
If you were to close out of this R session, the spark context would be destroyed and YARN would kill spark applicaiton. Alternately, you can stop the spark application and continue to work in the R session.

In [ ]:
sparkR.stop()

# Verify Spark-On-YARN Application is Finished:
Check the status of your application, named **SparkR**, within the [YARN All Applications](http://localhost:8088/cluster) List.